In [17]:
import tarfile
import json
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import os

INPUT_TAR = "./beem_models_data"
OUTPUT_JSONL = "promela_finetune.jsonl"

In [49]:
def parse_beem_xml(xml_text):
    summary = ""
    instruction_parts = []

    root = ET.fromstring(xml_text)

    # short-description → summary
    sd = root.find(".//short-description")
    if sd is not None and sd.text:
        summary = sd.text.strip()

    # long-description → instruction
    ld = root.find(".//long-description")
    if ld is not None and ld.text:
        instruction_parts.append(ld.text.strip())

    # parameters
    params = root.findall(".//parameter-description")
    if params:
        param_texts = []
        for p in params:
            if p.text:
                param_texts.append(p.text.strip())
        if param_texts:
            instruction_parts.append(
                "Parameters:\n" + "\n".join(param_texts)
            )

    # atomic propositions
    aps = root.findall(".//ap")
    if aps:
        ap_list = [ap.text.strip() for ap in aps if ap.text]
        if ap_list:
            instruction_parts.append(
                "Atomic propositions:\n" + ", ".join(ap_list)
            )

    # properties (LTL/CTL)
    props = root.findall(".//property")
    if props:
        prop_list = [p.text.strip() for p in props if p.text]
        if prop_list:
            instruction_parts.append(
                "Properties to verify:\n" + "\n".join(prop_list)
            )

    instruction = "\n\n".join(instruction_parts)

    return summary, instruction

In [21]:
for file in os.listdir(INPUT_TAR):
    print(file)

adding
anderson
at
bakery
blocks
bopdp
bridge
brp
brp2
cambridge
collision
cyclic_scheduler
driving_phils
elevator
elevator2
elevator_planning
exit
extinction
firewire_link
firewire_tree
fischer
frogs
gear
hanoi
iprotocol
krebs
lamport
lamport_nonatomic
lann
leader_election
leader_filters
lifts
loyd
lup
mcs
msmie
needham
peg_solitaire
peterson
pgm_protocol
phils
plc
pouring
production_cell
protocols
public_subscribe
reader_writer
resistance
rether
rushhour
schedule_world
sokoban
sorter
synapse
szymanski
telephony
train-gate


In [3]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path

# ================== CONFIG ==================
BEEM_ROOT = "beem_models_data"          # thư mục BEEM đã giải nén
OUTPUT_JSONL = "beem_promela.jsonl"
# ============================================


# ---------------- XML PARSER ----------------
def parse_beem_xml(xml_path: Path):
    summary = ""
    instruction_parts = []

    try:
        root = ET.parse(xml_path).getroot()

        def get_text(tag):
            el = root.find(f".//{tag}")
            return el.text.strip() if el is not None and el.text else ""

        summary = get_text("short-description")

        long_desc = get_text("long-description")
        if long_desc:
            instruction_parts.append(long_desc)

        params = [p.text.strip() for p in root.findall(".//parameter-description") if p.text]
        if params:
            instruction_parts.append("Parameters:\n" + "\n".join(params))

        aps = [a.text.strip() for a in root.findall(".//ap") if a.text]
        if aps:
            instruction_parts.append("Atomic propositions:\n" + ", ".join(aps))

        props = [p.text.strip() for p in root.findall(".//property") if p.text]
        if props:
            instruction_parts.append("Properties to verify:\n" + "\n".join(props))

    except Exception as e:
        print(f"⚠️ XML parse error: {xml_path} ({e})")

    return summary, "\n\n".join(instruction_parts)


# ---------------- DATASET BUILDER ----------------
def build_dataset(beem_root: Path):
    samples = []

    for model_dir in beem_root.iterdir():
        if not model_dir.is_dir():
            continue

        # XML
        xml_files = list(model_dir.glob("*.xml"))
        if not xml_files:
            continue

        summary, instruction = parse_beem_xml(xml_files[0])

        # PROMELA (.pm)
        pm_files = list((model_dir / "generated_files").glob("*.pm"))
        if not pm_files:
            continue

        for pm in pm_files:
            promela_code = pm.read_text(encoding="utf-8", errors="ignore").strip()

            if not promela_code:
                continue

            samples.append({
                "summary": summary,
                "instruction": instruction or f"Explain and verify the Promela model for {model_dir.name}.",
                "response": promela_code
            })

    return samples


# ---------------- MAIN ----------------
beem_root = Path(BEEM_ROOT)
dataset = build_dataset(beem_root)

with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
    for sample in dataset:
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

print(f"✅ Generated {len(dataset)} samples → {OUTPUT_JSONL}")


✅ Generated 235 samples → beem_promela.jsonl


In [1]:
# SYSTEM_PROMT = "
# You are an expert in Formal Verification, Model Checking, and PROMELA/SPIN.

# Your task is to transform a natural language software specification into a
# formally correct PROMELA model using a rigorous, multi-stage methodology.

# You MUST follow all stages below in order.
# Do NOT skip stages.
# Use precise formal reasoning and bounded state modeling.
# STAGE 1 – Specification Analysis

# From the natural language specification:

# 1. Identify ACTORS (entities that perform actions).
#    - Map each actor to a PROMELA proctype or active proctype.

# 2. Identify DATA ENTITIES:
#    - Shared data → global variables or channels.
#    - Local data → local variables inside proctype.
#    - Specify type: bool, byte, int, mtype, chan.

# 3. Identify STATE DESCRIPTORS:
#    - Adjectives or phrases describing states
#      (e.g., RED, GREEN, WAITING, LOCKED).
#    - Collect them as the candidate state set S.

# Output this stage as structured lists.
# STAGE 2 – State Modeling

# 1. Identify STATE VARIABLES:
#    - Variables that define the global or local system state.
#    - Specify initial values.

# 2. Define the STATE SPACE:
#    - Enumerate all valid values of state variables.
#    - Use mtype enumerations to bound the state space.

# 3. For each process:
#    - Define its control states (FSM states).
#    - Assign symbolic names (e.g., IDLE, GREEN, YELLOW, RED).

# Output:
# - State variables and domains
# - Control states per process

# STAGE 3 – Transition Logic

# 1. Identify EVENTS:
#    - Actions that trigger transitions
#      (e.g., timeout, button press, message receive).

# 2. Identify GUARDS:
#    - Conditions required for transitions.
#    - Express as boolean expressions.

# 3. Construct STATE TRANSITION TABLES:
#    For each process, list transitions in the form:
#    (Current State, Guard, Action, Next State)

# 4. Ensure:
#    - Deterministic or nondeterministic behavior is explicit.
#    - No implicit transitions.

# STAGE 4 – PROMELA Mapping

# 1. Translate each process FSM into PROMELA:
#    - Use do..od loops.
#    - Use :: guard -> action syntax.
#    - Represent states using mtype or explicit labels.

# 2. Use atomic or d_step:
#    - Only for logically indivisible transitions.
#    - To reduce state explosion where appropriate.

# 3. Define:
#    - Global variables
#    - Channels (if any)
#    - init process to start the system

# STAGE 5 – Verification

# 1. Identify SAFETY PROPERTIES:
#    - Conditions that must NEVER be violated
#      (e.g., never two conflicting traffic lights are GREEN).

# 2. Express each property as an LTL formula.

# 3. Optionally define:
#    - Monitor processes
#    - Assertions

# OUTPUT STRUCTURE (STRICT):

# STEP 1 – Global Variables, Channels, and Processes
# STEP 2 – State Space and Domains
# STEP 3 – Process FSM Descriptions
# STEP 4 – State Transition Tables
# STEP 5 – Safety Properties and LTL Formulas
# STEP 6 – Complete PROMELA Code

# The final PROMELA code must be:
# - Syntactically correct
# - Directly compilable by SPIN
# - Faithful to the original specification
# "

In [11]:
import os
import json
from tqdm import tqdm

# ===== CONFIG =====
INPUT_DIR = "beem_promela.jsonl"          # thư mục chứa file BEEM đã parse
OUTPUT_FILE = "beem_promela_finetune.jsonl"  # file output
MAX_CODE_CHARS = 20000           # optional, có thể bỏ nếu muốn

SYSTEM_PROMPT = """You are an expert in Formal Verification, Model Checking, and PROMELA/SPIN.

Internally apply a rigorous multi-stage methodology:
- Specification analysis and entity extraction
- State variable identification
- Finite-state modeling
- Transition table construction
- PROMELA mapping
- Safety verification using LTL

These stages are INTERNAL and MUST NOT be included in the output.

ONLY output valid, compilable PROMELA code.
Do NOT include explanations or stage descriptions, Do NOT use FSM, state, transition tables, or pseudo-code..
"""


In [13]:
def convert_beem_sample(data):
    instruction = data.get("instruction", "").strip()
    response = data.get("response", "").strip()

    if not instruction or not response:
        return None

    if MAX_CODE_CHARS and len(response) > MAX_CODE_CHARS:
        return None

    return {
        "messages": [
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": instruction
            },
            {
                "role": "assistant",
                "content": response
            }
        ]
    }


In [1]:
import json
from tqdm import tqdm

# ===== CONFIG =====
INPUT_JSONL = "beem_promela.jsonl"   # file JSONL đầu vào
OUTPUT_FILE = "beem_promela_finetune.jsonl"    # file output
MAX_CODE_CHARS = 20000             # optional
SYSTEM_PROMPT = """You are an expert in Formal Verification, Model Checking, and PROMELA/SPIN.

Internally apply a rigorous multi-stage methodology:
- Specification analysis and entity extraction
- State variable identification
- Finite-state modeling
- Transition table construction
- PROMELA mapping
- Safety verification using LTL

These stages are INTERNAL and MUST NOT be included in the output.

ONLY output valid, compilable PROMELA code.
Do NOT include explanations or stage descriptions, Do NOT use FSM, state, transition tables, or pseudo-code..
"""



def convert_beem_sample(data):
    instruction = data.get("instruction", "").strip()
    response = data.get("response", "").strip()

    if not instruction or not response:
        return None

    if MAX_CODE_CHARS and len(response) > MAX_CODE_CHARS:
        return None

    return {
        "messages": [
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": instruction
            },
            {
                "role": "assistant",
                "content": response
            }
        ]
    }


def main():
    valid, skipped = 0, 0

    with open(INPUT_JSONL, "r", encoding="utf-8") as fin, \
         open(OUTPUT_FILE, "w", encoding="utf-8") as fout:

        for line in tqdm(fin):
            line = line.strip()
            if not line:
                continue

            try:
                data = json.loads(line)
                sample = convert_beem_sample(data)

                if sample is None:
                    skipped += 1
                    continue

                fout.write(json.dumps(sample, ensure_ascii=False) + "\n")
                valid += 1

            except Exception:
                skipped += 1

    print("✅ DONE")
    print(f"✔ Valid samples : {valid}")
    print(f"✖ Skipped       : {skipped}")


if __name__ == "__main__":
    main()


235it [00:00, 11314.62it/s]

✅ DONE
✔ Valid samples : 228
✖ Skipped       : 7
